In [1]:
import pandas as pd
import sqlalchemy
import math
import os, sys, copy, random, pickle, cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append("../")

%load_ext autoreload
%autoreload 2

from modules.ControlTracking import TrackFactory
from modules.ControlDisplay import Display
from modules.Interproc import DBInterface, GuiviewState

from modules.AnalysisHelpers import ( 
         exploreImgs, buildImgComparisonData, subprocEval)

from modules.EvalHelpers import EvalTracker, EvalDataset
from modules.ControlEval import EvalFactory
from modules.DataSchemas import ScoreSchema
from modules.EvalHelpers import OutcomeData, DFHelper

from modules.EvalHelpers import AggEval, DFHelper

from modules.EvalSuite import EvalSuite

### Use subproc with logging

In [2]:
tmp_db_path = 'data/misc/books/eval-module-4/evaldb.db'

vid_path = 'data/proc/tmp/dec14/output5.proc1.proc1.avi'

outcome_subproc = subprocEval(f_pathfn = vid_path
                              ,db_pathfn = tmp_db_path 
                              ,b_log = True
                              ,b_eval_log = True
                             )


START - running eval module...
[                            ]
-----------------------------
output db: data/misc/books/eval-module-4/evaldb.db
output tbl: outcome_dataframe
eval time: 5.980
outcome rows: 345 cols: 48 

num frames:                  345
obj enums scored/tracked:    0,1 / 0
num scored frames:           19 | 0,9,78...308,324,340
-------
agg_checkBaselineInsideTrack       0.42
agg_checkBothContainsOther         0.42
agg_checkEitherContainsOther       0.53
agg_checkTrackInsideBaseline       0.53
agg_checkTrackInsideBaselineRect   0.11
agg_checkTrackSuccess              1.00
agg_compareRadii                   2.32
agg_distanceFromBaseline           5.49
subproc time: 8.059


### Use EvalSuite

In [30]:
from modules.EvalSuite import EvalSuite

In [31]:
suite = EvalSuite()
suite.buildFromOutcome(outcome_subproc)
suite.displayCli()


num frames:                  345
obj enums scored/tracked:    0,1 / 0
num scored frames:           19 | 0,9,78...308,324,340
-------
agg_calcBaselineBallUnitsAway       0.46
agg_checkBaselineInsideTrack        0.42
agg_checkBothContainsOther          0.42
agg_checkEitherContainsOther        0.53
agg_checkTrackInsideBaseline        0.53
agg_checkTrackInsideBaselineRect    0.11
agg_checkTrackSuccess               1.00
agg_compareRadii                    2.32
agg_distanceFromBaseline            5.49
agg_propBaselineRadius              9.84
agg_propTrackRadius                12.38


In [32]:
suite.previewEachDf(clip_rows=3)

agg_calcBaselineBallUnitsAway       0.46
agg_checkBaselineInsideTrack        0.42
agg_checkBothContainsOther          0.42
agg_checkEitherContainsOther        0.53
agg_checkTrackInsideBaseline        0.53
agg_checkTrackInsideBaselineRect    0.11
agg_checkTrackSuccess               1.00
agg_compareRadii                    2.32
agg_distanceFromBaseline            5.49
agg_propBaselineRadius              9.84
agg_propTrackRadius                12.38


,calc Baseline Ball Units Away,check Both Contains Other,distance From Baseline,check Track Success,prop Track Radius,check Baseline Inside Track,check Either Contains Other,check Track Inside Baseline,check Track Inside Baseline Rect,compare Radii,prop Baseline Radius
0,0.04,True,2,True,32,True,True,True,False,6,26.0
1,nan,None,nan,None,32,None,None,None,None,nan,NaN
2,nan,None,nan,None,31,None,None,None,None,nan,NaN


,index,input _data0 _0,input _data0 _1,input _data0 _2,input _data0 _3,input _data1 _0,input _data1 _1,input _data1 _2,input _data1 _3,input _data2 _0,...,track _data3 _2,track _data3 _3,track _obj _exists _0,track _obj _exists _1,track _obj _exists _2,track _obj _exists _3,track _obj _type _0,track _obj _type _1,track _obj _type _2,track _obj _type _3
0,0,107.0,111.0,None,None,214.0,218.0,None,None,54.0,...,None,None,True,None,None,None,circle,None,None,None
1,1,NaN,NaN,None,None,NaN,NaN,None,None,NaN,...,None,None,True,None,None,None,circle,None,None,None
2,2,NaN,NaN,None,None,NaN,NaN,None,None,NaN,...,None,None,True,None,None,None,circle,None,None,None


### Adding Properties + Calcs to Eval()

In [33]:
from modules.EvalSuite import EvalSuite
from modules.EvalHelpers import DFHelper, OutcomeData

In [34]:
suite = EvalSuite()
suite.buildFromOutcome(outcome_subproc)

In [35]:
eval_df = DFHelper(suite.evalTable)
eval_df.setRowsRequested(s_cmd='inputframes')
eval_df.getDatasetDisplay()

,calc Baseline Ball Units Away,check Both Contains Other,distance From Baseline,check Track Success,prop Track Radius,check Baseline Inside Track,check Either Contains Other,check Track Inside Baseline,check Track Inside Baseline Rect,compare Radii,prop Baseline Radius
0,0.04,True,2,True,32,True,True,True,False,6,26.0
9,0.14,True,7,True,33,True,True,True,False,7,26.0
78,0.03,True,1,True,24,True,True,True,False,-1,25.0
116,0.12,True,4,True,22,True,True,True,False,3,19.0
136,0.14,True,4,True,17,True,True,True,False,2,15.0
176,0.45,True,5,True,6,True,True,True,False,0,6.0
187,0.67,False,5,True,3,False,False,False,False,-1,4.0
194,0.75,False,6,True,2,False,False,False,False,-2,4.0
195,0.50,False,4,True,3,False,False,False,False,-1,4.0
201,0.52,False,4,True,1,False,False,False,False,-3,4.0
